In [171]:
using JuMP, Cbc
o = Model(with_optimizer(Cbc.Optimizer))

┌ Warning: `with_optimizer` is deprecated. Adapt the following example to update your code:
│ `with_optimizer(Ipopt.Optimizer)` becomes `Ipopt.Optimizer`.
│   caller = top-level scope at In[171]:2
└ @ Core In[171]:2


A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: COIN Branch-and-Cut (Cbc)

In [172]:
#viewers in millions in hours 9:21
v = [281.7
422
423
376.8
377.8
518.1
528
529
549.6
858.5
859.5
860.5
861.5]

13-element Array{Float64,1}:
 281.7
 422.0
 423.0
 376.8
 377.8
 518.1
 528.0
 529.0
 549.6
 858.5
 859.5
 860.5
 861.5

In [169]:
# qualifications variables
@variable(o, m_qual1[1:2, 9:21], Bin)      #men's qualification subdivision 1
@variable(o, m_qual2[1:2, 9:21], Bin)      #men's qualification subdivision 2
@variable(o, m_qual3[1:2, 9:21], Bin)      #men's qualification subdivision 3
@variable(o, w_qual1[1:2, 9:21], Bin)      #women's qualification subdivision 1
@variable(o, w_qual2[1:2, 9:21], Bin)      #women's qualification subdivision 2
@variable(o, w_qual3[1:2, 9:21], Bin)      #women's qualification subdivision 3

2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, Base.OneTo(2)
    Dimension 2, 9:21
And data, a 2×13 Array{VariableRef,2}:
 w_qual3[1,9]  w_qual3[1,10]  w_qual3[1,11]  …  w_qual3[1,20]  w_qual3[1,21]
 w_qual3[2,9]  w_qual3[2,10]  w_qual3[2,11]     w_qual3[2,20]  w_qual3[2,21]

In [170]:
# QUALIFIERS
# all mens qual rounds on either day 1 or 2 all womens qual rounds on either day 1 or 2
@variable(o, w, Bin)    
@constraint(o, sum(m_qual1[1,:]) + sum(m_qual2[1,:]) + sum(m_qual3[1,:]) == 6-(6*w))
@constraint(o, sum(m_qual1[2,:]) + sum(m_qual2[2,:]) + sum(m_qual3[2,:]) == 6*w)
@constraint(o, sum(w_qual1[1,:]) + sum(w_qual2[1,:]) + sum(w_qual3[1,:]) == 6*w)
@constraint(o, sum(w_qual1[2,:]) + sum(w_qual2[2,:]) + sum(w_qual3[2,:]) == 6-(6*w))

#the events cannot take place at the same time
for day in 1:2
    for hour in 9:21
        @constraint(o, sum(m_qual1[day, hour] + m_qual2[day, hour] + m_qual3[day, hour]) <= 1)
        @constraint(o, sum(w_qual1[day, hour] + w_qual2[day, hour] + w_qual3[day, hour]) <= 1)
    end
end

#each event takes two hours (the hours are not consecutive yet)/we might not need this if you guys think the part below is fine
all_qual = [m_qual1, m_qual2, m_qual3, w_qual1, w_qual2, w_qual3]
for event in all_qual
    @constraint(o, sum(event[1,:]) == 2-2*w)
    @constraint(o, sum(event[2,:]) == 2*w)
end

#each session takes two hours, and there are 2 hours in between

#hours are consecutive now for m_qual1
@variable(o, t_m1_1[9:20], Bin) #binary variable denoting the start time of m_qual1 on day 1
@variable(o, t_m1_2[9:20], Bin) #binary variable denoting the start time of m_qual1 on day 2
@constraint(o, sum(t_m1_1[:]) == w)
@constraint(o, sum(t_m1_2[:]) == 1-w)
for hour in 9:20
    @constraint(o, sum(m_qual1[1, hour]+ m_qual1[1, hour+1]) == 2*t_m1_1[hour])
    @constraint(o, sum(m_qual1[2, hour]+ m_qual1[2, hour+1]) == 2*t_m1_2[hour])
end

#hours are consecutive now for m_qual2
@variable(o, t_m2_1[9:20], Bin) #binary variable denoting the start time of m_qual2 on day 1
@variable(o, t_m2_2[9:20], Bin) #binary variable denoting the start time of m_qual2 on day 2
@constraint(o, sum(t_m2_1[:]) == w)
@constraint(o, sum(t_m2_2[:]) == 1-w)
for hour in 9:20
    @constraint(o, sum(m_qual2[1, hour]+ m_qual2[1, hour+1]) == 2*t_m2_1[hour])
    @constraint(o, sum(m_qual2[2, hour]+ m_qual2[2, hour+1]) == 2*t_m2_2[hour])
end

#hours are consecutive now for m_qual3
@variable(o, t_m3_1[9:20], Bin) #binary variable denoting the start time of m_qual3 on day 1
@variable(o, t_m3_2[9:20], Bin) #binary variable denoting the start time of m_qual3 on day 2
@constraint(o, sum(t_m3_1[:]) == w)
@constraint(o, sum(t_m3_2[:]) == 1-w)
for hour in 9:20
    @constraint(o, sum(m_qual3[1, hour]+ m_qual3[1, hour+1]) == 2*t_m3_1[hour])
    @constraint(o, sum(m_qual3[2, hour]+ m_qual3[2, hour+1]) == 2*t_m3_2[hour])
end

#hours are consecutive now for w_qual1
@variable(o, t_w1_1[9:20], Bin) #binary variable denoting the start time of w_qual1 on day 1
@variable(o, t_w1_2[9:20], Bin) #binary variable denoting the start time of w_qual1 on day 2
@constraint(o, sum(t_w1_1[:]) == 1-w)
@constraint(o, sum(t_w1_2[:]) == w)
for hour in 9:20
    @constraint(o, sum(w_qual1[1, hour]+ w_qual1[1, hour+1]) == 2*t_w1_1[hour])
    @constraint(o, sum(w_qual1[2, hour]+ w_qual1[2, hour+1]) == 2*t_w1_2[hour])
end

#hours are consecutive now for w_qual2
@variable(o, t_w2_1[9:20], Bin) #binary variable denoting the start time of w_qual2 on day 1
@variable(o, t_w2_2[9:20], Bin) #binary variable denoting the start time of w_qual2 on day 2
@constraint(o, sum(t_w2_1[:]) == 1-w)
@constraint(o, sum(t_w2_2[:]) == w)
for hour in 9:20
    @constraint(o, sum(w_qual2[1, hour]+ w_qual2[1, hour+1]) == 2*t_w2_1[hour])
    @constraint(o, sum(w_qual2[2, hour]+ w_qual2[2, hour+1]) == 2*t_w2_2[hour])
end

#hours are consecutive now for w_qual3
@variable(o, t_w3_1[9:20], Bin) #binary variable denoting the start time of w_qual3 on day 1
@variable(o, t_w3_2[9:20], Bin) #binary variable denoting the start time of w_qual3 on day 2
@constraint(o, sum(t_w3_1[:]) == 1-w)
@constraint(o, sum(t_w3_2[:]) == w)
for hour in 9:20
    @constraint(o, sum(w_qual3[1, hour]+ w_qual3[1, hour+1]) == 2*t_w3_1[hour])
    @constraint(o, sum(w_qual3[2, hour]+ w_qual3[2, hour+1]) == 2*t_w3_2[hour])
end

In [26]:
o

A JuMP Model
Feasibility problem with:
Variables: 157
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 40 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 40 constraints
`VariableRef`-in-`MathOptInterface.ZeroOne`: 157 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: COIN Branch-and-Cut (Cbc)
Names registered in the model: h, m_qual1, m_qual2, m_qual3, w_qual1, w_qual2, w_qual3

In [173]:
# TEAMS FINALS
@variable(o, m_tf[4:5, 9:21], Bin)
@variable(o, w_tf[4:5, 9:21], Bin)

#team finals constraints
# men and women on different days
@variable(o, x, Bin)
@constraint(o, sum(m_tf[4, :]) == 2*x)
@constraint(o, sum(m_tf[5, :]) == 2-(2*x))
@constraint(o, sum(w_tf[4, :]) == 2-(2*x))
@constraint(o, sum(w_tf[5, :]) == 2*x)

# 2 hr for men
@variable(o, t_mt_4[9:20], Bin)     #binary variable denoting the start time of men's team finals on day 4
@variable(o, t_mt_5[9:20], Bin)     #binary variable denoting the start time of men's team finals on day 5

@constraint(o, sum(t_mt_4[:]) == x) 
@constraint(o, sum(t_mt_5[:]) == 1-x)
#makes sure the hours are consecutive
for hour in 9:20
    @constraint(o, sum(m_tf[4, hour]+ m_tf[4, hour+1]) == 2*t_mt_4[hour])
    @constraint(o, sum(m_tf[5, hour]+ m_tf[5, hour+1]) == 2*t_mt_5[hour])
end

#2hrs for women
@variable(o, t_wt_4[9:20], Bin)     #binary variable denoting the start time of women's team finals on day 4
@variable(o, t_wt_5[9:20], Bin)     #binary variable denoting the start time of women's team finals on day 5
@constraint(o, sum(t_wt_4[:]) == 1-x) 
@constraint(o, sum(t_wt_5[:]) == x)
#makes sure the hours are consecutive
for hour in 9:20
    @constraint(o, sum(w_tf[4, hour]+ w_tf[4, hour+1]) == 2*t_wt_4[hour])
    @constraint(o, sum(w_tf[5, hour]+ w_tf[5, hour+1]) == 2*t_wt_5[hour])
end

In [174]:
# INDIVIDUAL ALL AROUND
@variable(o, m_ar[6:7, 9:21])
@variable(o, w_ar[6:7, 9:21])

# men and women on different days
@variable(o, d, Bin)
@constraint(o, sum(m_ar[6, :]) == 3*d)
@constraint(o, sum(m_ar[7, :]) == 3-(3*d))
@constraint(o, sum(w_ar[6, :]) == 2-(2*d))
@constraint(o, sum(w_ar[7, :]) == 2*d)

# individual constraints
#2 hrs for women
@variable(o, t_war_6[9:20], Bin)   #binary variable denoting the start time of women's ind. all around on day 6
@variable(o, t_war_7[9:20], Bin)   #binary variable denoting the start time of women's ind. all around on day 7
@constraint(o, sum(t_war_6[:]) == 1-d)
@constraint(o, sum(t_war_7[:]) == d)
#make sure the hours are two in a row
for hour in 9:20
    @constraint(o, sum(w_ar[6, hour]+ w_ar[6, hour+ 1]) == 2*t_war_6[hour])
    @constraint(o, sum(w_ar[7, hour]+ w_ar[7, hour+ 1]) == 2*t_war_7[hour])
end
                
#3hrs for men
@variable(o, t_mar_6[9:20], Bin)   #binary variable denoting the start time of men's ind. all around on day 6
@variable(o, t_mar_7[9:20], Bin)   #binary variable denoting the start time of men's ind. all around on day 7
@constraint(o, sum(t_mar_6[:]) == d)
@constraint(o, sum(t_mar_7[:]) == 1-d)
#make sure the 3 hours are in a row
for hour in 9:19
    @constraint(o, sum(m_ar[6, hour]+ m_ar[6, hour+ 1] + m_ar[6, hour+2]) == 3*t_mar_6[hour])
    @constraint(o, sum(m_ar[7, hour]+ m_ar[7, hour+ 1] + m_ar[7, hour+2]) == 3*t_mar_7[hour])
end

In [175]:
o

A JuMP Model
Feasibility problem with:
Variables: 202
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 110 constraints
`VariableRef`-in-`MathOptInterface.ZeroOne`: 150 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: COIN Branch-and-Cut (Cbc)
Names registered in the model: d, m_ar, m_tf, t_mar_6, t_mar_7, t_mt_4, t_mt_5, t_war_6, t_war_7, t_wt_4, t_wt_5, w_ar, w_tf, x

In [17]:
# specific events
@variable(o, m_floor[9:11, 9:21])      # mens floor
@variable(o, m_vault[9:11, 9:21])      # mens vault
@variable(o, m_pb[9:11, 9:21])         # mens parallel bars
@variable(o, m_pommel[9:11, 9:21])     # mens pommel horse
@variable(o, m_rings[9:11, 9:21])      # mens rings
@variable(o, m_horizontal[9:11, 9:21]) #mens horizontal bar
@variable(o, w_floor[9:11, 9:21])      # womens floor
@variable(o, w_vault[9:11, 9:21])      # womens vault
@variable(o, w_ub[9:11, 9:21])         # womens uneven bars
@variable(o, w_b[9:11, 9:21])          # womens beam


@constraint(o, sum(m_floor[9,i]+m_vault[9,i]+m_pb[9,i]+m_pommel[9,i]+m_rings[9,i]+m_horizontal[9,i]+ w_floor[9,i]+w_vault[9,i]+w_ub[9,i]+ w_b[9,i] for i in 9:21) >=3)
@constraint(o, sum(m_floor[9,i]+m_vault[9,i]+m_pb[9,i]+m_pommel[9,i]+m_rings[9,i]+m_horizontal[9,i]+ w_floor[9,i]+w_vault[9,i]+w_ub[9,i]+ w_b[9,i] for i in 9:21) <=4)

@constraint(o, sum(m_floor[10,i]+m_vault[10,i]+m_pb[10,i]+m_pommel[10,i]+m_rings[10,i]+m_horizontal[10,i]+ w_floor[10,i]+w_vault[10,i]+w_ub[10,i]+ w_b[10,i] for i in 9:21) >=3)
@constraint(o, sum(m_floor[10,i]+m_vault[10,i]+m_pb[10,i]+m_pommel[10,i]+m_rings[10,i]+m_horizontal[10,i]+ w_floor[10,i]+w_vault[10,i]+w_ub[10,i]+ w_b[10,i] for i in 9:21) <=4)

@constraint(o, sum(m_floor[11,i]+m_vault[11,i]+m_pb[11,i]+m_pommel[11,i]+m_rings[11,i]+m_horizontal[11,i]+ w_floor[11,i]+w_vault[11,i]+w_ub[11,i]+ w_b[11,i] for i in 9:21) >=3)
@constraint(o, sum(m_floor[11,i]+m_vault[11,i]+m_pb[11,i]+m_pommel[11,i]+m_rings[11,i]+m_horizontal[11,i]+ w_floor[11,i]+w_vault[11,i]+w_ub[11,i]+ w_b[11,i] for i in 9:21) <=4)


#at most two specific events can happen in an hour
#for each day of specific events, the sum of events in an hour cannot be more than 2
for i = 9:21
    @constraint(o, (m_floor[9,i]+m_vault[9,i]+m_pb[9,i]+m_pommel[9,i]+m_horizontal[9,i]+ w_floor[9,i]+w_vault[9,i]+w_ub[9,i]+ w_b[9,i]) <=2)
end

for i = 9:21
    @constraint(o, (m_floor[10,i]+m_vault[10,i]+m_pb[10,i]+m_pommel[10,i]+m_horizontal[10,i]+ w_floor[10,i]+w_vault[10,i]+w_ub[10,i]+ w_b[10,i]) <=2)
end

for i = 9:21
    @constraint(o, (m_floor[11,i]+m_vault[11,i]+m_pb[11,i]+m_pommel[11,i]+m_horizontal[11,i]+ w_floor[11,i]+w_vault[11,i]+w_ub[11,i]+ w_b[11,i]) <=2)
    end

#if womens floor is happening, mens floor cannot be happening (there is only one floor)
for i = 9:11
    for j = 9:21
        @constraint(o, (m_floor[i,j]+ w_floor[i,j) <=1))
    end
end




ErrorException: An object of name m_floor is already attached to this model. If this is intended, consider using the anonymous construction syntax, e.g., x = @variable(model, [1:N], ...) where the name of the object does not appear inside the macro.

In [ ]:
#add up viewers at possible times of events to get objective to be maximized
#mens qual
objec = v'*(m_qual1[1,:])+v'*(m_qual1[2,:])+ v'*(m_qual2[1,:])+v'*(m_qual2[2,:])+ v'*(m_qual3[1,:])+v'*(m_qual3[2,:])
#womens qual
objec = objec + v'*(w_qual1[1,:])+v'*(w_qual1[2,:])+ v'*(w_qual2[1,:])+v'*(w_qual2[2,:])+ v'*(w_qual3[1,:])+v'*(w_qual3[2,:])
#team finals
objec = objec +v'*m_tf[4,:]+v'*m_tf[5,:]+v'*w_tf[4,:]+v'*w_tf[5,:]
#indiv all around
objec = objec +v'*m_ar[6,:]+v'*m_ar[7,:]+v'*w_ar[6,:]+v'*w_ar[7,:]
#specific events
#always a day of rest before finals (day 8)
objec = objec +sum(v'*m_floor[i,:] for i in 9:11)
objec = objec +sum(v'*m_vault[i,:] for i in 9:11)
objec = objec +sum(v'*m_pb[i,:] for i in 9:11)
objec = objec +sum(v'*m_pommel[i,:] for i in 9:11)
objec = objec +sum(v'*m_horizontal[i,:] for i in 9:11)
objec = objec +sum(v'*m_rings[i,:] for i in 9:11)
objec = objec +sum(v'*w_floor[i,:] for i in 9:11)
objec = objec +sum(v'*w_vault[i,:] for i in 9:11)
objec = objec +sum(v'*w_ub[i,:] for i in 9:11)
objec = objec +sum(v'*w_b[i,:] for i in 9:11)


#objective

#@objective(o, Max,  objec)


#take a look at our model
#o

#optimize!(o)